## This notebook will be used for the Coursera Applied Data Science Capstone Project

In [5]:
!pip install pandas

Looking in indexes: http://localhost:3141/bv/dev/
     |████████████████████████████████| 10.6 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 21.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/josko374/.pyenv/versions/3.6.8/bin/python3.6 -m pip install --upgrade pip' command.


In [6]:
import pandas as pd
import numpy as np

In [7]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
